# **odoo**

# **Import Library**

In [0]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from  plotly.offline import plot
import itertools


# **Import File**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
try:
  df_one = pd.read_csv('/content/drive/My Drive/work/nectec/dataset7.csv',parse_dates=['Date'])
  with pd.ExcelFile('/content/drive/My Drive/work/nectec/dataSet4 (2)-newFromITaaS.xlsx') as reader:
    # print(reader.sheet_names)
    df_two = pd.read_excel(reader, sheet_name=reader.sheet_names[1])
except:
  df_one = pd.read_csv('/content/drive/My Drive/Nectec/dataset7.csv',parse_dates=['Date'])
  with pd.ExcelFile('/content/drive/My Drive/Nectec/dataSet4 (2)-newFromITaaS.xlsx') as reader:
    # print(reader.sheet_names)
    df_two = pd.read_excel(reader, sheet_name=reader.sheet_names[1])
# export_csv = df_one.to_csv('test_dataset7.csv')
# export_csv = df_two.to_csv('test_dataset4.csv')

In [0]:
df_all_columns = pd.merge(df_one, df_two, on='Item No', how='inner')
df_all_columns['sum_price'] = df_all_columns['Price'] * df_all_columns['Quantity']
df_clean_all = df_all_columns[['Receipt No','Item No','Price','Quantity','Date','Description','sum_price']]
df_clean_all

,Receipt No,Item No,Price,Quantity,Date,Description,sum_price
0,140050000,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0
1,140050221,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0
2,80004023,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0
3,80004023,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0
4,130031218,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0
...,...,...,...,...,...,...,...
981540,30020755,2751012A,40.0,1.0,2018-09-30,น้ำพริกตาแดงเห็ด3อย่าง(เจ)-แย้มเหน่อ,40.0
981541,40068855,2751012A,40.0,1.0,2018-09-30,น้ำพริกตาแดงเห็ด3อย่าง(เจ)-แย้มเหน่อ,40.0
981542,40068655,3412173A,1250.0,1.0,2018-09-30,เสื้อลินินคอปาดผูกโบว์ชาย-ริมนที,1250.0
981543,40068655,3412173A,1250.0,1.0,2018-09-30,เสื้อลินินคอปาดผูกโบว์ชาย-ริมนที,1250.0


In [0]:
df_sum_item = df_clean_all.groupby(['Item No','Date']).sum().reset_index()
df_sum_item = df_sum_item[['Receipt No','Item No','Date','sum_price','Quantity']]
df_sum_item
# df_sum_item['Description'] = df_clean_all['Description'].tolist()


,Receipt No,Item No,Date,sum_price,Quantity
0,94097602587,1111001A,2018-01-01,110320.0,3152.0
1,124086775957,1111001A,2018-01-02,126140.0,3604.0
2,37836447721,1111001A,2018-01-03,36715.0,1049.0
3,22617610866,1111001A,2018-01-04,23800.0,680.0
4,17664752529,1111001A,2018-01-05,22225.0,635.0
...,...,...,...,...,...
127927,80134040,3999004A,2018-09-07,516.0,4.0
127928,130044853,3999004A,2018-09-22,129.0,1.0
127929,80100758,3999005A,2018-03-23,358.0,2.0
127930,140073841,3999005A,2018-09-21,358.0,2.0


In [0]:
# export_csv = df_sum_item.to_csv('clean_datav2.csv')
# export_csv

# **Function**

In [0]:
# step 1เลือกรายได้หรือปริมาณ
df_choose_item = df_sum_item
# print('--------------------------------------------------------------------------------------------------------------------')
def choose_income_quality(choose_income_quality):
  # print(choose)
  df_choose_item = df_sum_item.sort_values(f"{choose_income_quality}",ascending=False)
  df_choose_item[f'total_{choose_income_quality}'] = df_choose_item[f'{choose_income_quality}'].sum()
  df_choose_item[f'percent_{choose_income_quality}'] = df_choose_item[f'{choose_income_quality}']/df_choose_item[f'total_{choose_income_quality}'] * 100
  df_choose_item[f'cumulative_{choose_income_quality}'] = df_choose_item[f'percent_{choose_income_quality}'].cumsum()
  return df_choose_item
df_choose_item = choose_income_quality('sum_price')
# df_choose_item = choose_income_quality('Quantity')

# print('--------------------------------------------------------------------------------------------------------------------')
# ทำcumulativeตัดที่กี่เปอร์เซ็น
def split_cumulative(percent):
  return df_choose_item[-int(len(df_choose_item) / 100 * percent):]
# df_choose_item = split_cumulative(10)
df_choose_item = split_cumulative(80)

# print('--------------------------------------------------------------------------------------------------------------------')
# เลือกวันที่
def between_date(start_date, end_date):
  return df_choose_item.loc[(df_choose_item['Date'] > start_date) & (df_choose_item['Date'] <= end_date)]
df_choose_item = between_date('2018-01-01','2018-12-12')
# print('--------------------------------------------------------------------------------------------------------------------')
# เลือกรายได้หรือปริมาณเเละเลือกจำนวนtopที่จะออกมา
def top_item(total_list,choose_income_quality):
    list_top = df_choose_item.groupby('Item No').sum().reset_index().sort_values(f'{choose_income_quality}',ascending=False)['Item No'].tolist()[:10]
    if total_list == 5:
      df_list_item = df_choose_item[(df_choose_item['Item No'] == list_top[0]) | 
                      (df_choose_item['Item No'] == list_top[1]) | 
                      (df_choose_item['Item No'] == list_top[2]) | 
                      (df_choose_item['Item No'] == list_top[3]) | 
                      (df_choose_item['Item No'] == list_top[4])]
    elif total_list == 10:
      df_list_item = df_choose_item[(df_choose_item['Item No'] == list_top[0]) | 
                      (df_choose_item['Item No'] == list_top[1]) | 
                      (df_choose_item['Item No'] == list_top[2]) | 
                      (df_choose_item['Item No'] == list_top[3]) | 
                      (df_choose_item['Item No'] == list_top[4]) | 
                      (df_choose_item['Item No'] == list_top[5]) | 
                      (df_choose_item['Item No'] == list_top[6]) |
                      (df_choose_item['Item No'] == list_top[7]) | 
                      (df_choose_item['Item No'] == list_top[8]) | 
                      (df_choose_item['Item No'] == list_top[9])]
    return df_list_item
# df_choose_item = top_item(10,'sum_price')
# df_choose_item = top_item(5,'Quantity')
df_choose_item

,Receipt No,Item No,Date,sum_price,Quantity,total_sum_price,percent_sum_price,cumulative_sum_price
89029,220103985,3255010A,2018-08-31,520.0,8.0,75222440.84,0.000691,78.232521
77940,400172696,3245010A,2018-06-18,520.0,4.0,75222440.84,0.000691,78.233213
40232,590348232,3211045A,2018-03-13,520.0,8.0,75222440.84,0.000691,78.233904
44111,651026450,3212040A,2018-02-03,520.0,13.0,75222440.84,0.000691,78.234595
77954,160241948,3245010A,2018-07-07,520.0,4.0,75222440.84,0.000691,78.235286
...,...,...,...,...,...,...,...,...
96232,160245842,3260025A,2018-07-20,0.0,0.0,75222440.84,0.000000,100.000000
28919,240069864,1640005A,2018-07-08,0.0,0.0,75222440.84,0.000000,100.000000
95690,180509464,3260016A,2018-03-08,0.0,0.0,75222440.84,0.000000,100.000000
124429,60041298,3621018A,2018-09-03,0.0,0.0,75222440.84,0.000000,100.000000


In [0]:
# df_all_choose = pd.merge(df_choose_item,df_clean_all[["Description","Receipt No"]],on='Receipt No')
# df_all_choose

# **Masket Basket**

In [0]:
df_basket = (df_choose_item
          .groupby(['Receipt No', 'Item No'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Receipt No'))
df_basket
# df_basket = (df_all_choose
#           .groupby(['Receipt No', 'Description'])['Quantity']
#           .sum().unstack().reset_index().fillna(0)
#           .set_index('Receipt No'))
# df_basket

Item No,1111001C,1111002A,1111002C,1111003A,1111003C,1111004A,1111004C,1111005C,1111009A,1111009C,1111010A,1111010C,1111011A,1111011C,1112007A,1113007A,1113009A,1113009C,1113012A,1113031A,1113048B,1113051A,1113078A,1113080A,1113089A,1113092A,1113093A,1113094A,1113095A,1113095C,1113096A,1113096C,1113101A,1113101C,1113102A,1113102C,1113120A,1113120B,1113120C,1113121A,...,3622047A,3622048A,3622050A,3622060A,3622066A,3622067A,3622077A,3622078A,3622080A,3622082A,3622085A,3622086A,3622087A,3622088A,3622089A,3624002A,3624008A,3624015A,3624026A,3624027A,3624028A,3624029A,3624030A,3629004A,3819012A,3819051A,3819052A,3912001A,3912006A,3916005A,3916019A,3916027A,3917002A,3917003A,3917004A,3917005A,3917006A,3999003A,3999004A,3999005A
Receipt No,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20011726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20011842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20011950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20011966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20012100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3804934877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3816999086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3925480321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
import datetime
time_start = datetime.datetime.now()
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = df_basket.applymap(encode_units)
time_end = datetime.datetime.now()
print(time_end - time_start)
basket_sets

0:00:45.411961


Item No,1111001C,1111002A,1111002C,1111003A,1111003C,1111004A,1111004C,1111005C,1111009A,1111009C,1111010A,1111010C,1111011A,1111011C,1112007A,1113007A,1113009A,1113009C,1113012A,1113031A,1113048B,1113051A,1113078A,1113080A,1113089A,1113092A,1113093A,1113094A,1113095A,1113095C,1113096A,1113096C,1113101A,1113101C,1113102A,1113102C,1113120A,1113120B,1113120C,1113121A,...,3622047A,3622048A,3622050A,3622060A,3622066A,3622067A,3622077A,3622078A,3622080A,3622082A,3622085A,3622086A,3622087A,3622088A,3622089A,3624002A,3624008A,3624015A,3624026A,3624027A,3624028A,3624029A,3624030A,3629004A,3819012A,3819051A,3819052A,3912001A,3912006A,3916005A,3916019A,3916027A,3917002A,3917003A,3917004A,3917005A,3917006A,3999003A,3999004A,3999005A
Receipt No,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20011726,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20011842,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20011950,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20011966,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20012100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3804934877,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3816999086,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3925480321,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
frequent_itemsets = apriori(basket_sets, min_support=0.0001, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.000423,(1111001C)
1,0.000245,(1111002C)
2,0.000412,(1111003C)
3,0.000345,(1111004C)
4,0.000267,(1111009A)
...,...,...
1231,0.000367,"(3917006A, 3917004A)"
1232,0.000111,"(3323060A, 3917006A, 3917004A)"
1233,0.000122,"(3323061A, 3917006A, 3917002A)"
1234,0.000145,"(3323129A, 3917003A, 3917005A)"


In [0]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1141070A),(1141069A),0.002673,0.002506,0.000145,0.054167,21.618759,0.000138,1.054620
1,(1141069A),(1141070A),0.002506,0.002673,0.000145,0.057778,21.618759,0.000138,1.058484
2,(1141075A),(1141069A),0.001258,0.002506,0.000100,0.079646,31.787965,0.000097,1.083816
3,(1141069A),(1141075A),0.002506,0.001258,0.000100,0.040000,31.787965,0.000097,1.040356
4,(1141070A),(1141072A),0.002673,0.001336,0.000145,0.054167,40.535174,0.000141,1.055856
...,...,...,...,...,...,...,...,...,...
163,"(3323134A, 3917004A)",(3917006A),0.000145,0.001748,0.000111,0.769231,439.985301,0.000111,4.325757
164,"(3917006A, 3917004A)",(3323134A),0.000367,0.000301,0.000111,0.303030,1007.867565,0.000111,1.434351
165,(3323134A),"(3917006A, 3917004A)",0.000301,0.000367,0.000111,0.370370,1007.867565,0.000111,1.587652
166,(3917006A),"(3323134A, 3917004A)",0.001748,0.000145,0.000111,0.063694,439.985301,0.000111,1.067873


In [0]:
rules['LHS'] = [",".join(tuple(rules['antecedents'][i])) for i in range(len(rules['antecedents']))]
rules['RHS'] = [",".join(tuple(rules['consequents'][i])) for i in range(len(rules['consequents']))]
df_market_basket = rules[['LHS','RHS','antecedent support','consequent support','support','confidence','lift','leverage','conviction']]
df_market_basket

,LHS,RHS,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,1141070A,1141069A,0.002673,0.002506,0.000145,0.054167,21.618759,0.000138,1.054620
1,1141069A,1141070A,0.002506,0.002673,0.000145,0.057778,21.618759,0.000138,1.058484
2,1141075A,1141069A,0.001258,0.002506,0.000100,0.079646,31.787965,0.000097,1.083816
3,1141069A,1141075A,0.002506,0.001258,0.000100,0.040000,31.787965,0.000097,1.040356
4,1141070A,1141072A,0.002673,0.001336,0.000145,0.054167,40.535174,0.000141,1.055856
...,...,...,...,...,...,...,...,...,...
163,"3323134A,3917004A",3917006A,0.000145,0.001748,0.000111,0.769231,439.985301,0.000111,4.325757
164,"3917006A,3917004A",3323134A,0.000367,0.000301,0.000111,0.303030,1007.867565,0.000111,1.434351
165,3323134A,"3917006A,3917004A",0.000301,0.000367,0.000111,0.370370,1007.867565,0.000111,1.587652
166,3917006A,"3323134A,3917004A",0.001748,0.000145,0.000111,0.063694,439.985301,0.000111,1.067873


In [0]:
# export_csv = df_market_basket.to_csv('masket_basketv2.csv')

# **Network X**

In [0]:
A = list(df_market_basket["LHS"].unique())
B = list(df_market_basket["RHS"].unique())
# print(list(set(A+B)))
node_list = list(set(A+B))
print(node_list)
G = nx.Graph()
for i,j in df_market_basket.iterrows():
    G.add_edges_from([(j["LHS"],j["RHS"])])
pos = nx.spring_layout(G, k=0.5, iterations=50)
for n, p in pos.items():
    G.nodes[n]['pos'] = p
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')
arrow_x0 = []
arrow_x1 = []
arrow_y0 = []
arrow_y1 = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])
    arrow_x0.append(x0)
    arrow_x1.append(x1)
    arrow_y0.append(y0)
    arrow_y1.append(y1)
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='RdBu',
        reversescale=True,
        color=[],
        size=15,
        colorbar=dict(
            thickness=10,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=0)))
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color']+=tuple([len(adjacencies[1])])
    node_info = adjacencies[0]
    node_trace['text']+=tuple([node_info])
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>NECTEC-ODOO',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                # annotations = [
                #     dict(ax=arrow_x0[i], ay=arrow_y0[i], axref='x', ayref='y',
                #         x=arrow_x1[i], y=arrow_y1[i], xref='x', yref='y',
                #         showarrow=True, arrowhead=1,) for i in range(len(arrow_x0))
                # ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
# plot(fig)
fig.show()

# nx.enumerate_all_cliques(G)

['3259029A', '3260035A', '3917006A,3917004A', '1142022A', '3617012A', '3617013A', '3617014A', '3541049A', '3263001A', '3323060A', '3250095A', '3917002A', '3219157A', '3250120A', '3917005A,3323129A', '3249008A', '3250013A', '3254006A', '3329146A', '3250119A', '3252011A', '3257126A', '3239048A', '3257127A', '3618001A', '3249009A', '3617016A', '3259028A', '3323128A', '3239055A', '3323135A', '3212020A', '3239003A', '3249011A', '3323134A', '3212018A', '3254005A', '3263004A', '3917006A', '3329148A', '3511022A', '3614107A', '3614109A', '3541047A', '3219158A', '3257128A', '3263003A', '2237015A', '3250092A', '3262002A', '3614111A', '3260017A', '3617015A', '3323061A,3917002A', '3323134A,3917006A', '3239001A', '3250015A', '3619001A', '3323129A', '3323061A', '3249010A', '3219140A', '3614106A', '3263005A', '3323127A', '1141069A', '3511020A', '3917004A', '3329147A', '3256001A', '1142020A', '3259027A', '3263002A', '3323060A,3917006A', '3917003A,3323129A', '2237014A', '1141075A', '3614110A', '1141072A

In [0]:
import networkx
from networkx.algorithms.approximation import clique
print('---------------------------------------------------------')
print('node_network_all')
print(node_list)
print('---------------------------------------------------------')
print('clique.max_clique(G)')
print(clique.max_clique(G))
print('---------------------------------------------------------')
print('clique.large_clique_size(G)')
print(clique.large_clique_size(G))
print('---------------------------------------------------------')
print('clique.clique_removal(G)')
print(clique.clique_removal(G))
print('-----------------------------------------------------------------------------')
print('nx.enumerate_all_cliques(G)')
print(nx.enumerate_all_cliques(G))
for i in nx.enumerate_all_cliques(G):
  print(i)
print('---------------------------------------------------------')
print('nx.find_cliques(G)')
print(nx.find_cliques(G))
for i in nx.find_cliques(G):
  print(i)
print('---------------------------------------------------------')
print('nx.make_max_clique_graph(G)')
print(nx.make_max_clique_graph(G))
for i in nx.make_max_clique_graph(G):
  print(nx.make_max_clique_graph(G)[i])
print('---------------------------------------------------------')
print('nx.make_clique_bipartite(G)')
print(nx.make_clique_bipartite(G))
for i in nx.make_clique_bipartite(G):
  print(i)
print('---------------------------------------------------------')
print('nx.graph_clique_number(G)')
print(nx.graph_clique_number(G))
print('---------------------------------------------------------')
print('nx.graph_number_of_cliques(G)')
print(nx.graph_number_of_cliques(G))
print('---------------------------------------------------------')
print('nx.node_clique_number(G)')
print(nx.node_clique_number(G))
print('---------------------------------------------------------')
print('nx.number_of_cliques(G)')
print(nx.number_of_cliques(G))
print('---------------------------------------------------------')
print('nx.cliques_containing_node(G)')
print(nx.cliques_containing_node(G))
print('---------------------------------------------------------')


---------------------------------------------------------
node_network_all
['3259029A', '3260035A', '3917006A,3917004A', '1142022A', '3617012A', '3617013A', '3617014A', '3541049A', '3263001A', '3323060A', '3250095A', '3917002A', '3219157A', '3250120A', '3917005A,3323129A', '3249008A', '3250013A', '3254006A', '3329146A', '3250119A', '3252011A', '3257126A', '3239048A', '3257127A', '3618001A', '3249009A', '3617016A', '3259028A', '3323128A', '3239055A', '3323135A', '3212020A', '3239003A', '3249011A', '3323134A', '3212018A', '3254005A', '3263004A', '3917006A', '3329148A', '3511022A', '3614107A', '3614109A', '3541047A', '3219158A', '3257128A', '3263003A', '2237015A', '3250092A', '3262002A', '3614111A', '3260017A', '3617015A', '3323061A,3917002A', '3323134A,3917006A', '3239001A', '3250015A', '3619001A', '3323129A', '3323061A', '3249010A', '3219140A', '3614106A', '3263005A', '3323127A', '1141069A', '3511020A', '3917004A', '3329147A', '3256001A', '1142020A', '3259027A', '3263002A', '3323060A,39

# **Clique Network X**

In [0]:
# print(clique.max_clique(G))
# print(clique.large_clique_size(G))
# print(clique.clique_removal(G))
# for i in range(len(clique.clique_removal(G)[0])):
#   print(list(clique.clique_removal(G)[0])[i])

df_market_basket['node'] = list(zip(df_market_basket['LHS'],  df_market_basket['RHS']))
# df
print()
import itertools
import networkx as nx
G = nx.Graph()
edges_fig_4 = df_market_basket['node'].tolist()
edges_fig_4
G.add_edges_from(edges_fig_4)
cliques = nx.find_cliques(G)
length_node = 3
cliques3 = set(sum([list(itertools.combinations(set(clq), length_node)) for clq in cliques if len(clq)>length_node],[]))
print(cliques3)
# print(list(cliques3))
df_market_basket['node'] 

set()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0               (1141070A, 1141069A)
1               (1141069A, 1141070A)
2               (1141075A, 1141069A)
3               (1141069A, 1141075A)
4               (1141070A, 1141072A)
                   ...              
163    (3323134A,3917004A, 3917006A)
164    (3917006A,3917004A, 3323134A)
165    (3323134A, 3917006A,3917004A)
166    (3917006A, 3323134A,3917004A)
167    (3917004A, 3323134A,3917006A)
Name: node, Length: 168, dtype: object

In [0]:
# A = list(df["LHS"].unique())
# B = list(df["RHS"].unique())
# node_list = list(set(A+B))
G = nx.Graph()
for i in range(len(list(cliques3))):
    G.add_edges_from([(list(cliques3)[i][0],list(cliques3)[i][1])])
pos = nx.spring_layout(G, k=0.5, iterations=50)
for n, p in pos.items():
    G.nodes[n]['pos'] = p
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')
arrow_x0 = []
arrow_x1 = []
arrow_y0 = []
arrow_y1 = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])
    arrow_x0.append(x0)
    arrow_x1.append(x1)
    arrow_y0.append(y0)
    arrow_y1.append(y1)
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Picnic',
        reversescale=True,
        color=[],
        size=15,
        colorbar=dict(
            thickness=10,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=0)))
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color']+=tuple([len(adjacencies[1])])
    node_info = adjacencies[0]
    node_trace['text']+=tuple([node_info])

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                # annotations = [
                #     dict(ax=arrow_x0[i], ay=arrow_y0[i], axref='x', ayref='y',
                #         x=arrow_x1[i], y=arrow_y1[i], xref='x', yref='y',
                #         showarrow=True, arrowhead=1,) for i in range(len(arrow_x0))
                # ],
                title='<br>NECTEC-ODOO-clique',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
fig.show()
# print(cliques.max_clique(G))
# print(cliques.large_clique_size(G))
# print(cliques.clique_removal(G))
# {('1111004A', '1111002A'), ('1111010A', '1111001A'), ('1111003A', '1111001A'), ('1111004A', '1111001A'), ('1111009A', '1111010A'), ('1111003A', '1111002A'), ('1111002A', '1111001A'), ('1111009A', '1111001A')}
print(cliques3)

set()


In [0]:
degree = nx.degree_centrality(G)
closeness = nx.closeness_centrality(G)
betweenness = nx.betweenness_centrality(G)
# for key,values in betweenness.items():
#   print(key,values)
df_degree = pd.DataFrame(list(degree.items()),columns=['node','degree'])
df_closeness = pd.DataFrame(list(closeness.items()),columns=['node','closeness'])
df_betweenness = pd.DataFrame(list(betweenness.items()),columns=['node','betweenness'])
# df
df_one = pd.merge(df_degree,df_closeness,on='node')
df_all = pd.merge(df_one,df_betweenness,on='node') 
df_all

,degree,closeness,node,betweenness


# **Dash Board**

# **Test Jack**

In [0]:
# How to run a Dash app in Google Colab

## Requirements

### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8049 &')

### Get the public URL where you can access the Dash app. Copy this URL.
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Install Dash
!pip install dash==0.31.1  # The core dash backend
!pip install dash-html-components==0.13.2  # HTML components
!pip install dash-core-components==0.39.0  # Supercharged components
!pip install dash-table==3.1.7  # Interactive DataTable component (new!)

--2020-01-09 05:19:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.238.36.128, 52.5.84.255, 34.231.143.21, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.238.36.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  14.3MB/s    in 0.9s    

2020-01-09 05:19:44 (14.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://a67fa903.ngrok.io
     |████████████████████████████████| 1.1MB 13.1MB/s 
  Created wheel for dash: filename=dash-0.31.1-cp36-none-any.whl size=36085 sha256=c24a2050ec64b7f75e004b6be2d3d543aca6ccdd5c19dc0a8a994b4991227cbf
  Stored in directory: /root/.cache/pip/wheels/50/0c/6d/3a89e4f4d6b11094a08d01ea33df5fea228bf0aee1ccc5d981
  Created w

In [0]:
## Dash app (https://dash.plot.ly/getting-started)

### Save file with Dash app on the Google Colab machine
%%writefile my_app1.py
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Writing my_app1.py


In [0]:
### Run Dash app
!python my_app1.py

Running on http://127.0.0.1:8050/
Debugger PIN: 311-738-272
 * Serving Flask app "my_app1" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Running on http://127.0.0.1:8050/
Debugger PIN: 147-244-426


In [0]:
/create data
var data = [
    {x: "A", value: 100},
    {x: "B", value: 100},
    {x: ["A", "B"], value: 25}
];

// create a chart and set the data
chart = anychart.venn(data);

// configure labels of intersections
chart.intersections().labels().format("{%x}");

// set the container id
chart.container('container');

// initiate drawing the chart
chart.draw();